Cleaning
- first to truncate the data by merging it with the flight data. then removing those data where the flight data is relatively non-eventful (we call this the setup stage, where the subject wears all the apparatus and get ready)
- next is to perform FFT to remove the noise. based on equipment specs (normal range should be between 10-30 hz?)

There could be 2 source of features from the PPG raw data that was collected
- heart beats/min every 20seconds (this is calculated by taking average time interval between each peak , take the average over every 20s, then take 20s divided by average time interval (eg 1s between each peak))
- amptitude. but this could be due to how strong the device is pressed against the finger. So we disregard this

In [1]:
import re, os, time
import pandas as pd
import datetime
import numpy as np
from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score
from matplotlib import pyplot as plt
# import plotData # helper function in starter code package


from sktime.transformations.panel.padder import PaddingTransformer
from sktime.classification.compose import ClassifierPipeline, ComposableTimeSeriesForestClassifier
from sktime.transformations.panel.summarize import RandomIntervalFeatureExtractor
from sklearn.tree import DecisionTreeClassifier
# only classifier in sktime that can process unequal length data
# https://github.com/sktime/sktime/issues/3649#issuecomment-1292459843
# from sktime.alignment.dtw_python import AlignerDTW   ## NOTE THAT THIS SOMEHOW AFFECT ALL PRINT OUTPUT. NOTHING WILL BE SHOWN FOR PRINT STATEMENT AFTER YOU RUN THIS
from sktime.classification.feature_based import RandomIntervalClassifier
from sktime.classification.distance_based import KNeighborsTimeSeriesClassifier
from sktime.classification.dictionary_based import IndividualBOSS, ContractableBOSS
from sktime.classification.kernel_based import RocketClassifier
from sktime.classification.hybrid import HIVECOTEV1, HIVECOTEV2
from sktime.classification.shapelet_based import ShapeletTransformClassifier
from sktime.classification.sklearn import RotationForest

from sktime.dists_kernels.compose_from_align import DistFromAligner
from sktime.utils.slope_and_trend import _slope
from sklearn.pipeline import Pipeline
# https://www.sktime.org/en/stable/api_reference/auto_generated/sktime.transformations.panel.catch22.Catch22.html
from sktime.transformations.panel.catch22 import Catch22

from sktime.classification.interval_based import CanonicalIntervalForest,DrCIF,RandomIntervalSpectralEnsemble,SupervisedTimeSeriesForest,TimeSeriesForestClassifier

# identify classifiers that support unequal length
from sktime.registry import all_estimators




In [ ]:
data_root = "..\cleanedDataPPG\"

## Preparing data from Xue Yang

In [2]:
df_ppg = pd.read_csv(data_root+"eda_ppg_window_20s.csv")

In [3]:
df_combined_subject_20s = pd.DataFrame()
df_combined_subject_20s["subject"] = df_ppg["group"].apply(lambda x: x.split("-")[0])
df_combined_subject_20s["difficulty"] = df_ppg["group"].apply(lambda x: x.split("-")[1])

In [4]:
df_combined_subject_20s["normalised_pulse"] = ""
for i in range(len(df_combined_subject_20s)):
    temp_series = pd.DataFrame(df_ppg.drop(['file_name', 'level_idx', 'level', 'level_0', 'level_1', 'level_2','level_3', 'level_4', 'group_idx', 'group'], axis = 1).iloc[i].dropna())
    # somehow we need to convert to array, flatten then back to sereis again for sktime to read it properly
    temp_series = pd.Series(np.array(temp_series).flatten())
    df_combined_subject_20s.at[i,"normalised_pulse"] = temp_series

In [5]:
# save to pickle rather than csv to preserve the nested series inside the dataframe
df_combined_subject_20s.to_pickle(data_root+"df_combined_subject_20s.pkl", protocol=4)

### scaled

In [6]:
df_combined_subject_20s_scaled = df_combined_subject_20s.copy()

In [7]:
df_combined_subject_20s_scaled["normalised_pulse"] = ""
for i in range(len(df_combined_subject_20s_scaled)):
    temp_series = pd.DataFrame(df_ppg.drop(['file_name', 'level_idx', 'level', 'level_0', 'level_1', 'level_2','level_3', 'level_4', 'group_idx', 'group'], axis = 1).iloc[i].dropna())
    scaler = StandardScaler()
    temp_series = pd.Series(scaler.fit_transform(temp_series).flatten())
    df_combined_subject_20s_scaled.at[i,"normalised_pulse"] = temp_series

In [8]:
df_combined_subject_20s_scaled.to_pickle(data_root+"df_combined_subject_20s_scaled.pkl", protocol=4)

### minmax

In [9]:
df_combined_subject_20s_minmax = df_combined_subject_20s.copy()

In [10]:
df_combined_subject_20s_minmax["normalised_pulse"] = ""
for i in range(len(df_combined_subject_20s_minmax)):
    temp_series = pd.DataFrame(df_ppg.drop(['file_name', 'level_idx', 'level', 'level_0', 'level_1', 'level_2','level_3', 'level_4', 'group_idx', 'group'], axis = 1).iloc[i].dropna())
    scaler = StandardScaler()
    temp_series = pd.Series(scaler.fit_transform(temp_series).flatten())
    df_combined_subject_20s_minmax.at[i,"normalised_pulse"] = temp_series

In [11]:
df_combined_subject_20s_minmax.to_pickle(data_root+"df_combined_subject_20s_minmax.pkl", protocol=4)

# Modelling NO scaling

In [12]:
df_combined_subject_20s = pd.read_pickle(data_root+"df_combined_subject_20s.pkl")

In [13]:
%%time
# perform train test split according by subject
# split into 5 different folds for CV
from sklearn.model_selection import GroupKFold
X_train, X_test, y_train, y_test = [], [], [], []
gss = GroupKFold(n_splits=5)
for train, test in gss.split(df_combined_subject_20s["normalised_pulse"], df_combined_subject_20s["difficulty"], df_combined_subject_20s["subject"]):
  X_train.append(df_combined_subject_20s.loc[train,["normalised_pulse"]])
  X_test.append(df_combined_subject_20s.loc[test,["normalised_pulse"]])
  y_train.append(df_combined_subject_20s.loc[train,"difficulty"].astype("string"))
  y_test.append(df_combined_subject_20s.loc[test,"difficulty"].astype("string"))

CPU times: total: 0 ns
Wall time: 17.6 ms


In [14]:
model_result = {
  "classifier":[],
  "accuracy_score":[],
  "AUC_score":[],
  "F1_score":[],
  "runtime(s)":[],
}

def get_class(class_list, prob_list):
  idx = list(prob_list).index(max(prob_list))
  return class_list[idx]

def log_result(classifier_name, class_list, y_test, y_pred_proba, runtime):
  y_pred = []
  for y_list in y_pred_proba:
    y_pred.append(get_class(class_list, y_list))
  acc = accuracy_score(y_test, y_pred)
  auc = roc_auc_score(y_test, y_pred_proba, multi_class='ovr')
  f1 = f1_score(y_test, y_pred, average='micro')
  model_result["classifier"].append(classifier_name)
  model_result["accuracy_score"].append(acc)
  model_result["AUC_score"].append(auc)
  model_result["F1_score"].append(f1)
  model_result["runtime(s)"].append(runtime)

  display(pd.DataFrame(model_result))
  pd.DataFrame(model_result).to_csv(data_root+"ppg_split_pilot_result.csv")

In [15]:
def run_model(classifier_name,classifier, padding = True,  fold = 0):
  start = time.time()
  
  # set up pipeline
  if padding:
    clf = PaddingTransformer() * classifier()
  else:
    clf = classifier()
  
  clf.fit(X_train[fold], y_train[fold])
  y_pred_proba = clf.predict_proba(X_test[fold])
  end = time.time()

  log_result(classifier_name, clf.classes_, y_test[fold], y_pred_proba, end-start)

## Classification using catch22

Refer to respiration_split_pilot_pycaret.ipynb

## RandomIntervalClassifier
extract at random interval and perform Rotation forest with 200 trees

In [16]:
run_model(
  "RandomIntervalClassifier", 
  lambda: RandomIntervalClassifier(n_intervals=5, n_jobs=1, random_state = 42),
  padding = True,
  fold = 0
  )

,classifier,accuracy_score,AUC_score,F1_score,runtime(s)
0,RandomIntervalClassifier,0.375,0.646991,0.375,13.032886


## Decision Trees with mean, std, slope


In [17]:
steps = [
    ("padding",PaddingTransformer()),
    (
        "extract",
        RandomIntervalFeatureExtractor(
            n_intervals="sqrt", features=[np.mean, np.std, _slope]
        ),
    ),
    ("clf", DecisionTreeClassifier()),
]
time_series_tree = Pipeline(steps)

In [18]:
start = time.time()
time_series_tree.fit(X_train[0], y_train[0])
y_pred_proba = time_series_tree.predict_proba(X_test[0])
end = time.time()
log_result('RandomeIntervalDecisionTree',time_series_tree.classes_, y_test[0], y_pred_proba, end-start)

,classifier,accuracy_score,AUC_score,F1_score,runtime(s)
0,RandomIntervalClassifier,0.375000,0.646991,0.375000,13.032886
1,RandomeIntervalDecisionTree,0.291667,0.527778,0.291667,2.108154


## Individual Boss


In [19]:
run_model(
  "IndividualBOSS", 
  lambda: IndividualBOSS(),
  padding = True,
  fold = 0
  )

,classifier,accuracy_score,AUC_score,F1_score,runtime(s)
0,RandomIntervalClassifier,0.375000,0.646991,0.375000,13.032886
1,RandomeIntervalDecisionTree,0.291667,0.527778,0.291667,2.108154
2,IndividualBOSS,0.222222,0.481481,0.222222,3.091046


## ContractableBoss

In [20]:
run_model(
  "ContractableBOSS", 
  lambda: ContractableBOSS(n_parameter_samples=10, max_ensemble_size=3),
  padding = True,
  fold = 0
  )

,classifier,accuracy_score,AUC_score,F1_score,runtime(s)
0,RandomIntervalClassifier,0.375000,0.646991,0.375000,13.032886
1,RandomeIntervalDecisionTree,0.291667,0.527778,0.291667,2.108154
2,IndividualBOSS,0.222222,0.481481,0.222222,3.091046
3,ContractableBOSS,0.305556,0.506816,0.305556,3.176938


## Random Interval Spectral Ensemble

In [21]:
run_model(
  "RandomIntervalSpectralEnsemble", 
  lambda: RandomIntervalSpectralEnsemble(n_estimators=50, random_state=42),
  padding = True,
  fold = 0
  )

,classifier,accuracy_score,AUC_score,F1_score,runtime(s)
0,RandomIntervalClassifier,0.375000,0.646991,0.375000,13.032886
1,RandomeIntervalDecisionTree,0.291667,0.527778,0.291667,2.108154
2,IndividualBOSS,0.222222,0.481481,0.222222,3.091046
3,ContractableBOSS,0.305556,0.506816,0.305556,3.176938
4,RandomIntervalSpectralEnsemble,0.277778,0.573174,0.277778,4.314614


## Supervised Time Series Forest (STSF)


In [22]:
run_model(
  "SupervisedTimeSeriesForest", 
  lambda: SupervisedTimeSeriesForest(n_estimators=50, random_state=42),
  padding = True,
  fold = 0
  )

,classifier,accuracy_score,AUC_score,F1_score,runtime(s)
0,RandomIntervalClassifier,0.375000,0.646991,0.375000,13.032886
1,RandomeIntervalDecisionTree,0.291667,0.527778,0.291667,2.108154
2,IndividualBOSS,0.222222,0.481481,0.222222,3.091046
3,ContractableBOSS,0.305556,0.506816,0.305556,3.176938
4,RandomIntervalSpectralEnsemble,0.277778,0.573174,0.277778,4.314614
5,SupervisedTimeSeriesForest,0.444444,0.705890,0.444444,8.031878


## Canonical Interval Forest (CIF)

In [23]:
run_model(
  "CanonicalIntervalForest", 
  lambda: CanonicalIntervalForest(n_estimators=5, att_subsample_size=10, random_state=42),
  padding = True,
  fold = 0
  )

,classifier,accuracy_score,AUC_score,F1_score,runtime(s)
0,RandomIntervalClassifier,0.375000,0.646991,0.375000,13.032886
1,RandomeIntervalDecisionTree,0.291667,0.527778,0.291667,2.108154
2,IndividualBOSS,0.222222,0.481481,0.222222,3.091046
3,ContractableBOSS,0.305556,0.506816,0.305556,3.176938
4,RandomIntervalSpectralEnsemble,0.277778,0.573174,0.277778,4.314614
5,SupervisedTimeSeriesForest,0.444444,0.705890,0.444444,8.031878
6,CanonicalIntervalForest,0.347222,0.606096,0.347222,11.574676


## Diverse Representation Canonical Interval Forest (DrCIF)

In [24]:
run_model(
  "DiverseRepresentationCanonicalIntervalForest", 
  lambda: DrCIF(n_estimators=5, att_subsample_size=10, random_state=42),
  padding = True,
  fold = 0
  )

,classifier,accuracy_score,AUC_score,F1_score,runtime(s)
0,RandomIntervalClassifier,0.375000,0.646991,0.375000,13.032886
1,RandomeIntervalDecisionTree,0.291667,0.527778,0.291667,2.108154
2,IndividualBOSS,0.222222,0.481481,0.222222,3.091046
3,ContractableBOSS,0.305556,0.506816,0.305556,3.176938
4,RandomIntervalSpectralEnsemble,0.277778,0.573174,0.277778,4.314614
5,SupervisedTimeSeriesForest,0.444444,0.705890,0.444444,8.031878
6,CanonicalIntervalForest,0.347222,0.606096,0.347222,11.574676
7,DiverseRepresentationCanonicalIntervalForest,0.291667,0.596579,0.291667,18.151806


## ShapeletTransformClassifier

In [25]:
run_model(
  "ShapeletTransformClassifier", 
  lambda: ShapeletTransformClassifier(
    estimator=RotationForest(n_estimators=3),
    n_shapelet_samples=100,
    max_shapelets=10,
    batch_size=20,
    ),
  padding = True,
  fold = 0
  )

,classifier,accuracy_score,AUC_score,F1_score,runtime(s)
0,RandomIntervalClassifier,0.375000,0.646991,0.375000,13.032886
1,RandomeIntervalDecisionTree,0.291667,0.527778,0.291667,2.108154
2,IndividualBOSS,0.222222,0.481481,0.222222,3.091046
3,ContractableBOSS,0.305556,0.506816,0.305556,3.176938
4,RandomIntervalSpectralEnsemble,0.277778,0.573174,0.277778,4.314614
5,SupervisedTimeSeriesForest,0.444444,0.705890,0.444444,8.031878
6,CanonicalIntervalForest,0.347222,0.606096,0.347222,11.574676
7,DiverseRepresentationCanonicalIntervalForest,0.291667,0.596579,0.291667,18.151806
8,ShapeletTransformClassifier,0.361111,0.574074,0.361111,4.634022


## RocketClassifier

In [26]:
run_model(
  "RocketClassifier", 
  lambda: RocketClassifier(num_kernels=500),
  padding = True,
  fold = 0
  )

,classifier,accuracy_score,AUC_score,F1_score,runtime(s)
0,RandomIntervalClassifier,0.375000,0.646991,0.375000,13.032886
1,RandomeIntervalDecisionTree,0.291667,0.527778,0.291667,2.108154
2,IndividualBOSS,0.222222,0.481481,0.222222,3.091046
3,ContractableBOSS,0.305556,0.506816,0.305556,3.176938
4,RandomIntervalSpectralEnsemble,0.277778,0.573174,0.277778,4.314614
5,SupervisedTimeSeriesForest,0.444444,0.705890,0.444444,8.031878
6,CanonicalIntervalForest,0.347222,0.606096,0.347222,11.574676
7,DiverseRepresentationCanonicalIntervalForest,0.291667,0.596579,0.291667,18.151806
8,ShapeletTransformClassifier,0.361111,0.574074,0.361111,4.634022
9,RocketClassifier,0.444444,0.629630,0.444444,5.205036


## FAILED MODELS

### KNeighborsTimeSeriesClassifier

In [40]:
from sktime.alignment.dtw_python import AlignerDTW
from sktime.dists_kernels.compose_from_align import DistFromAligner

start = time.time()
aligner = AlignerDTW()
dtw_dist = DistFromAligner(aligner)
knclassifier = KNeighborsTimeSeriesClassifier(n_neighbors=3, distance = dtw_dist, n_jobs= -1)
knclassifier.fit(X_train[0], y_train[0])
y_pred_proba = knclassifier.predict_proba(X_test[0])
end = time.time()

log_result('KNeighborsTimeSeriesClassifier_AlignerDTW', knclassifier.classes_ ,y_test[0], y_pred_proba, end-start)


,classifier,accuracy_score,AUC_score,F1_score,runtime(s)
0,RandomIntervalClassifier,0.375000,0.646991,0.375000,13.032886
1,RandomeIntervalDecisionTree,0.291667,0.527778,0.291667,2.108154
2,IndividualBOSS,0.222222,0.481481,0.222222,3.091046
3,ContractableBOSS,0.305556,0.506816,0.305556,3.176938
4,RandomIntervalSpectralEnsemble,0.277778,0.573174,0.277778,4.314614
5,SupervisedTimeSeriesForest,0.444444,0.705890,0.444444,8.031878
6,CanonicalIntervalForest,0.347222,0.606096,0.347222,11.574676
7,DiverseRepresentationCanonicalIntervalForest,0.291667,0.596579,0.291667,18.151806
8,ShapeletTransformClassifier,0.361111,0.574074,0.361111,4.634022
9,RocketClassifier,0.444444,0.629630,0.444444,5.205036


### KNeighborsTimeSeriesClassifier with padding

In [41]:
run_model(
  "KNeighborsTimeSeriesClassifier_padded", 
  lambda: KNeighborsTimeSeriesClassifier(n_neighbors=5, distance ="dtw", n_jobs= 1, leaf_size = 2000),
  padding = True,
  fold = 0
  )

,classifier,accuracy_score,AUC_score,F1_score,runtime(s)
0,RandomIntervalClassifier,0.375000,0.646991,0.375000,13.032886
1,RandomeIntervalDecisionTree,0.291667,0.527778,0.291667,2.108154
2,IndividualBOSS,0.222222,0.481481,0.222222,3.091046
3,ContractableBOSS,0.305556,0.506816,0.305556,3.176938
4,RandomIntervalSpectralEnsemble,0.277778,0.573174,0.277778,4.314614
5,SupervisedTimeSeriesForest,0.444444,0.705890,0.444444,8.031878
6,CanonicalIntervalForest,0.347222,0.606096,0.347222,11.574676
7,DiverseRepresentationCanonicalIntervalForest,0.291667,0.596579,0.291667,18.151806
8,ShapeletTransformClassifier,0.361111,0.574074,0.361111,4.634022
9,RocketClassifier,0.444444,0.629630,0.444444,5.205036


### ComposableTimeSeriesForestClassifier
https://www.sktime.org/en/v0.8.1/examples/02_classification_univariate.html

In [43]:
run_model(
  "ComposableTimeSeriesForestClassifier", 
  lambda: ComposableTimeSeriesForestClassifier(
    estimator=time_series_tree,
    n_estimators=100,
    bootstrap=True,
    oob_score=True,
    random_state=1,
    n_jobs=-1,
    ),
  padding = True,
  fold = 0
  )

,classifier,accuracy_score,AUC_score,F1_score,runtime(s)
0,RandomIntervalClassifier,0.375000,0.646991,0.375000,13.032886
1,RandomeIntervalDecisionTree,0.291667,0.527778,0.291667,2.108154
2,IndividualBOSS,0.222222,0.481481,0.222222,3.091046
3,ContractableBOSS,0.305556,0.506816,0.305556,3.176938
4,RandomIntervalSpectralEnsemble,0.277778,0.573174,0.277778,4.314614
5,SupervisedTimeSeriesForest,0.444444,0.705890,0.444444,8.031878
6,CanonicalIntervalForest,0.347222,0.606096,0.347222,11.574676
7,DiverseRepresentationCanonicalIntervalForest,0.291667,0.596579,0.291667,18.151806
8,ShapeletTransformClassifier,0.361111,0.574074,0.361111,4.634022
9,RocketClassifier,0.444444,0.629630,0.444444,5.205036


### HIVECOTEV1

In [44]:
run_model(
  "HIVECOTEV1", 
  lambda: HIVECOTEV1(),
  padding = True,
  fold = 0
  )

# STOPPED PREMATURELY BECAUSE NO OUTPUT EVEN AFTER 24 hours

KeyboardInterrupt: 

# Modelling Scaled

In [ ]:
scaled_model_name = [
    'RandomIntervalClassifier_scaled',
    'IndividualBOSS_scaled',
    'ContractableBOSS_scaled',
    "RandomIntervalSpectralEnsemble_scaled", 
    "SupervisedTimeSeriesForest_scaled",
    "CanonicalIntervalForest_scaled", 
    "DiverseRepresentationCanonicalIntervalForest_scaled", 
    "ShapeletTransformClassifier_scaled", 
    "RocketClassifier_scaled", 
    "KNeighborsTimeSeriesClassifier_padded_scaled", 
    "ComposableTimeSeriesForestClassifier_scaled", 
    "HIVECOTEV1_scaled", 
    ]
scaled_model=[
    RandomIntervalClassifier(n_intervals=5, n_jobs=1, random_state = 42),
    IndividualBOSS(),
    ContractableBOSS(n_parameter_samples=10, max_ensemble_size=3),
    RandomIntervalSpectralEnsemble(n_estimators=50, random_state=42),
    SupervisedTimeSeriesForest(n_estimators=50, random_state=42),
    CanonicalIntervalForest(n_estimators=5, att_subsample_size=10, random_state=42),
    DrCIF(n_estimators=5, att_subsample_size=10, random_state=42),
    ShapeletTransformClassifier(estimator=RotationForest(n_estimators=3),n_shapelet_samples=100,max_shapelets=10,batch_size=20,),
    RocketClassifier(num_kernels=500),
    KNeighborsTimeSeriesClassifier(n_neighbors=5, distance ="dtw", n_jobs= 1, leaf_size = 2000),
    ComposableTimeSeriesForestClassifier(estimator=time_series_tree,n_estimators=100,bootstrap=True,oob_score=True,random_state=1,n_jobs=-1),
    HIVECOTEV1(),
    ]

In [ ]:
for i in len(scaled_model):
    run_model(
        scaled_model_name[i], 
        lambda: scaled_model[i],
        padding = True,
        fold = 0
        )


# Modelling MinMax

In [ ]:
minmax_model_name = [
    'RandomIntervalClassifier_minmax',
    'IndividualBOSS_minmax',
    'ContractableBOSS_minmax',
    "RandomIntervalSpectralEnsemble_minmax", 
    "SupervisedTimeSeriesForest_minmax",
    "CanonicalIntervalForest_minmax", 
    "DiverseRepresentationCanonicalIntervalForest_minmax", 
    "ShapeletTransformClassifier_minmax", 
    "RocketClassifier_minmax", 
    "KNeighborsTimeSeriesClassifier_padded_minmax", 
    "ComposableTimeSeriesForestClassifier_minmax", 
    "HIVECOTEV1_minmax", 
    ]
minmax_model=[
    RandomIntervalClassifier(n_intervals=5, n_jobs=1, random_state = 42),
    IndividualBOSS(),
    ContractableBOSS(n_parameter_samples=10, max_ensemble_size=3),
    RandomIntervalSpectralEnsemble(n_estimators=50, random_state=42),
    SupervisedTimeSeriesForest(n_estimators=50, random_state=42),
    CanonicalIntervalForest(n_estimators=5, att_subsample_size=10, random_state=42),
    DrCIF(n_estimators=5, att_subsample_size=10, random_state=42),
    ShapeletTransformClassifier(estimator=RotationForest(n_estimators=3),n_shapelet_samples=100,max_shapelets=10,batch_size=20,),
    RocketClassifier(num_kernels=500),
    KNeighborsTimeSeriesClassifier(n_neighbors=5, distance ="dtw", n_jobs= 1, leaf_size = 2000),
    ComposableTimeSeriesForestClassifier(estimator=time_series_tree,n_estimators=100,bootstrap=True,oob_score=True,random_state=1,n_jobs=-1),
    HIVECOTEV1(),
    ]

In [ ]:
for i in len(minmax_model):
    run_model(
        minmax_model_name[i], 
        lambda: minmax_model[i],
        padding = True,
        fold = 0
        )


# Optimising SupervisedTimeSeriesForest

In [32]:
asd

NameError: name 'asd' is not defined

In [ ]:
for i in np.arange(10,300,20):
    run_model(
        "SupervisedTimeSeriesForest_"+str(i)+"_trees", 
        lambda: SupervisedTimeSeriesForest(n_estimators=i, random_state=42),
        padding = True,
        fold = 0
    )

KeyboardInterrupt: 

# Try looking
- filter out different subject with different resp hz then do training for them.
- do gridsearch using genetic to improve result